In [ ]:
######### NOVO CÓDIGO 333333333333333

In [32]:
import pandas as pd
from datetime import date, timedelta, datetime
import re

# Define a data de ontem
hoje = date.today()
#ontem = date(2025, 10, 21)

print(f"Filtrando registros para: {hoje.strftime('%d/%m/%Y')}")


Filtrando registros para: 27/10/2025


In [33]:
# import re # Certifique-se de ter importado a biblioteca 're'
#
# arquivos = ['Ponto_Algodoeira.txt', 'Ponto_Escritorio.txt', 'Ponto_Sede.txt', 'Ponto_Secador.txt']
# dfs_ponto = []
#
# for arquivo in arquivos:
#
#     df = pd.read_csv(arquivo, header=None, names=['linha_completa'], encoding='latin-1')
#     df['sequencia'] = df['linha_completa'].str.slice(0, 10)
#     df['data'] = pd.to_datetime(df['linha_completa'].str.slice(10, 18), format='%d%m%Y', errors='coerce')
#     df['hora'] = pd.to_datetime(df['linha_completa'].str.slice(18, 22), format='%H%M', errors='coerce').dt.time
#     df['data_hora'] = pd.to_datetime(
#         df['data'].dt.strftime('%d/%m/%Y') + ' ' + df['hora'].astype(str),
#         format='%d/%m/%Y %H:%M',
#         errors='coerce'
#     )
#     #    0001524263 01072024 0657 021207043801adb7
#
#     #df['NIT'] = df['linha_completa'].apply(lambda linha: re.findall(r'\d{11}', linha)[-1] if re.findall(r'\d{11}', linha) else None)
#     #df['NIT'] = df['linha_completa'].apply(lambda linha: re.findall(r'[1-9]\d{10}', linha)[-1] if re.findall(r'[1-9]\d{10}', linha) else None)
#
#     # ... (extrações de data e hora)
#
#     # Função para extrair o NIT com base em ser o segundo padrão de 10/11 dígitos
#     def extrair_nit_segundo_padrao(linha):
#         # A regex busca o primeiro número (\d{10,11}), qualquer coisa (.*?), e CAPTURA o segundo
#         match = re.search(r'\d{10,11}.*?(\d{10,11})', linha)
#
#         if match:
#             # Retorna o segundo número encontrado
#             return match.group(1)
#
#         # Caso só haja um único número (ex: o arquivo só tem o NIT)
#         nits_encontrados = re.findall(r'\d{10,11}', linha)
#         if len(nits_encontrados) == 1:
#             return nits_encontrados[0]
#
#         return None
#
#     df['NIT'] = df['linha_completa'].apply(extrair_nit_segundo_padrao)
#
#     # ... (criação da coluna 'origem' e append)
#
# # ... (restante do código)
#     df['origem'] = arquivo
#     dfs_ponto.append(df)
#
#
# # Junta todos os arquivos processados
# df_ponto = pd.concat(dfs_ponto, ignore_index=True)
#

import pandas as pd
# Não é necessário importar 're' para esta solução

arquivos = ['Ponto_Algodoeira.txt', 'Ponto_Escritorio.txt', 'Ponto_Sede.txt', 'Ponto_Secador.txt']
dfs_ponto = []

# --- 1. Loop para processar cada arquivo ---
for arquivo in arquivos:

    # 1.1. Leitura do arquivo CSV (ou TXT)
    # header=None: indica que o arquivo não tem cabeçalho
    # names=['linha_completa']: dá nome à única coluna
    df = pd.read_csv(arquivo, header=None, names=['linha_completa'], encoding='latin-1')

    # 1.2. Extração de campos baseados em posição fixa (já validados)
    df['sequencia'] = df['linha_completa'].str.slice(0, 10)
    df['data'] = pd.to_datetime(df['linha_completa'].str.slice(10, 18), format='%d%m%Y', errors='coerce')
    df['hora'] = pd.to_datetime(df['linha_completa'].str.slice(18, 22), format='%H%M', errors='coerce').dt.time

    # 1.3. Combinação de Data e Hora
    df['data_hora'] = pd.to_datetime(
        df['data'].dt.strftime('%d/%m/%Y') + ' ' + df['hora'].astype(str),
        format='%d/%m/%Y %H:%M',
        errors='coerce'
    )

    # 🌟 1.4. SOLUÇÃO CORRIGIDA PARA EXTRAÇÃO DO NIT (10 OU 11 DÍGITOS) 🌟
    # O NIT começa na posição 24 e tem até 11 dígitos (termina na 35).

    # Primeiro, fatiamos a área do NIT
    nit_candidato = df['linha_completa'].str.slice(24, 35)

    # Segundo, limpamos o valor, removendo QUALQUER caractere que não seja dígito.
    # Isso garante que se o NIT for de 10 dígitos (ex: "1234567890 "), o espaço seja removido.
    # E que o NIT de 11 dígitos seja mantido intacto ("21207043801").
    df['NIT'] = nit_candidato.str.replace(r'[^\d]', '', regex=True)

    # 1.5. Adiciona a origem e coleta o DataFrame
    df['origem'] = arquivo
    dfs_ponto.append(df)

# --- 2. Junta todos os arquivos processados ---
df_ponto = pd.concat(dfs_ponto, ignore_index=True)

# Exibe as primeiras linhas do resultado para verificação
print(df_ponto[['linha_completa', 'data_hora', 'NIT']].head())

# Fim do código completo.


                                      linha_completa data_hora          NIT
0  00000000012   19018223972370700014182WALDEMIR ...       NaT  23707000141
1  000000001401012001000005022019111200000000000ae6f       NaT  02201911120
2                           000000002606022019113806       NaT             
3                           000000003606022019113902       NaT             
4                           000000004606022019115702       NaT             


In [34]:
# Fixar a data
data_alvo = hoje

# Filtrar registros da data
df_dia = df_ponto[df_ponto['data'].dt.date == data_alvo]

# Contar quantos registros cada NIT teve
contagem = df_dia['NIT'].value_counts()

# Filtrar apenas NITs com um único registro
nits_unicos = contagem[contagem == 1].index

# Salvar em arquivo .txt
nome_arquivo_nits = 'nits_unicos.txt'

try:
    with open(nome_arquivo_nits, 'w', encoding='utf-8') as f:
        f.write(f"NITs com ponto único em {data_alvo.strftime('%d/%m/%Y')}\n")
        f.write("=" * 40 + "\n\n")
        for nit in nits_unicos:
            f.write(f"{nit}\n")
    print(f"\n✓ Arquivo gerado com sucesso: {nome_arquivo_nits}")
except Exception as e:
    print(f"\n✗ Erro ao salvar arquivo: {e}")



✓ Arquivo gerado com sucesso: nits_unicos.txt


In [35]:
# Fixar a data
data_alvo = hoje

# Filtrar registros da data
df_dia = df_ponto[df_ponto['data'].dt.date == data_alvo]

# Contar quantos registros cada NIT teve
contagem = df_dia['NIT'].value_counts()

# Filtrar apenas NITs com um único registro
nits_unicos = contagem[contagem == 1].index

# Filtrar DataFrame com esses NITs
df_unico = df_dia[df_dia['NIT'].isin(nits_unicos)]

# Contar por origem
resultado = df_unico['origem'].value_counts()
print(resultado)


origem
Ponto_Sede.txt          49
Ponto_Algodoeira.txt    46
Ponto_Escritorio.txt    29
Ponto_Secador.txt        7
Name: count, dtype: int64


In [36]:

# # Carrega os NITs únicos do arquivo de texto
# with open('nits_unicos.txt', 'r', encoding='latin-1') as f:
#     nits_batidos = set(line.strip() for line in f if line.strip())
#
# # Carrega os dados dos funcionários do Excel
# df_funcionarios = pd.read_excel(r'Funcionarios.xlsx')
#
# # Filtra os funcionários que bateram o ponto
# df_batidos = df_funcionarios[df_funcionarios['NIT'].astype(str).isin(nits_batidos)]
#
# # Exibe os nomes e seções dos funcionários que bateram o ponto
# resultado = df_batidos[['Nome', 'Secao']]
# print(resultado)

#import pandas as pd
import io # Importamos 'io' para usar um String Buffer na hora de salvar, melhorando a organização

# --- 1. CARREGAMENTO DOS DADOS ---

# Carrega os NITs únicos do arquivo de texto
with open('nits_unicos.txt', 'r', encoding='latin-1') as f:
    # Usamos set() para garantir busca rápida e apenas valores únicos
    nits_batidos = set(line.strip() for line in f if line.strip())

# Carrega os dados dos funcionários do Excel
# É uma boa prática usar raw string (r'...') em caminhos de arquivo no Windows
df_funcionarios = pd.read_excel(r'Funcionarios.xlsx')

# --- 2. FILTRAGEM DOS DADOS ---

# Filtra os funcionários que bateram o ponto
# Converte a coluna 'NIT' para string antes de verificar se está no conjunto
df_batidos = df_funcionarios[df_funcionarios['NIT'].astype(str).isin(nits_batidos)]

# --- 3. AGRUPAMENTO E FORMATAÇÃO ---

# Seleciona apenas as colunas de interesse
resultado_df = df_batidos[['Nome', 'Secao']]

# Ordena e Agrupa pela 'Secao'
# O sort_values é opcional, mas ajuda a garantir a ordem dentro da seção
resultado_df = resultado_df.sort_values(by=['Secao', 'Nome'])

# Cria um objeto buffer para salvar o texto formatado
output_buffer = io.StringIO()

# Itera sobre os grupos e salva no buffer
# `name` será o nome da Seção, e `group` será o DataFrame com os funcionários daquela seção
for secao, grupo in resultado_df.groupby('Secao'):
    # Escreve o nome da seção como um cabeçalho
    output_buffer.write(f"\n========================================\n")
    output_buffer.write(f"SEÇÃO: {secao}\n")
    output_buffer.write(f"TOTAL DE FUNCIONÁRIOS NA SEÇÃO: {len(grupo)}\n")
    output_buffer.write(f"========================================\n")

    # Escreve a lista de nomes
    for nome in grupo['Nome']:
        output_buffer.write(f"- {nome}\n")
    output_buffer.write("\n")


# --- 4. SALVAMENTO NO ARQUIVO .TXT ---

# Abre o arquivo para escrita e salva o conteúdo do buffer
with open('Funcionarios_que_Bateram_Ponto.txt', 'w', encoding='utf-8') as f:
    f.write(output_buffer.getvalue())

# Exibe mensagem de sucesso
print("\n--- Processamento Concluído ---")
print("Os funcionários que bateram o ponto, agrupados por Seção, foram salvos em:")
print("Funcionarios_que_Bateram_Ponto.txt")


--- Processamento Concluído ---
Os funcionários que bateram o ponto, agrupados por Seção, foram salvos em:
Funcionarios_que_Bateram_Ponto.txt


In [ ]:
nits_batidos  